#Introdução
O ambiente Colab, por padrão, compila e executa códigos em python. Entretanto, vamos utilizá-lo para executar programas em linguagem "C" com uso de uma biblioteca MPI. Devemos ter certeza que os pacotes necessários estão instalados. A principio, os seguintes pacotes serão necessários:

- gcc
- openmpi

Altere o tipo de ambiente de execução para v2-8 TPU. Em caso de dúvidas utilize o apoio do assistente de IA Gemini.


In [ ]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!mpicc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!sudo apt install openmpi-doc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  openmpi-doc
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 4,268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openmpi-doc all 4.1.2-2ubuntu1 [1,072 kB]
Fetched 1,072 kB in 0s (4,686 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package openmpi-doc.
(Reading database ... 123632 files 

Em seguida, vamos importar os arquivos a serem utilizados no curso do repositório https://github.com/Programacao-Paralela-e-Distribuida/MPI . Depois organizar os arquivos adequadamente para nossos exemplos.

In [ ]:
!git clone https://github.com/Programacao-Paralela-e-Distribuida/MPI

Cloning into 'MPI'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 142 (delta 8), reused 5 (delta 5), pack-reused 120 (from 1)
Receiving objects: 100% (142/142), 3.94 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [ ]:
!mv MPI/* .
!rm -rf MPI
!ls -l

total 36
drwxr-xr-x 2 root root  4096 Jan 14 14:27 bin
drwxr-xr-x 2 root root  4096 Jan 14 14:27 docs
-rw-r--r-- 1 root root   547 Jan 14 14:27 Makefile
-rw-r--r-- 1 root root 13256 Jan 14 14:27 README.md
drwxr-xr-x 1 root root  4096 Jan  9 14:19 sample_data
drwxr-xr-x 2 root root  4096 Jan 14 14:27 src


#Funções auxiliares
No exemplo a seguir, fazemos uso de algumas funções de serviço do MPI, como por exemplo, **MPI_Abort()** para abortar um programa;  **MPI_Get_version()** para
identificar a versão do MPI; **MPI_Get_processor_name()** para recuperar o nome do computador.

A função **MPI_Init()** permite que o sistema realize as operações de preparação necessárias para que a biblioteca MPI seja utilizada.
Ao término do programa a função **MPI_Finalize()** deve ser chamada.  

Neste exemplo, a função **MPI_Comm_Rank()** retorna o *ranque* (rank) de um processo no seu segundo argumento.  O número de processos executando o programa pode ser obtido com a função **MPI_Comm_size()**.

In [ ]:
!cat src/mpi_funcoes.c

#include <stdio.h>
#include "mpi.h"

int main(int argc, char *argv[]) { /* mpi_funcoes.c  */
int meu_ranque, num_procs; 
int versao, subversao,  aux, ret;
char maquina[MPI_MAX_PROCESSOR_NAME];
    /* Inicia o MPI. Em caso de erro aborta o programa */
    ret = MPI_Init(&argc, &argv);
    if (ret != MPI_SUCCESS) {
        printf("Erro ao iniciar o programa MPI. Abortando.\n");
        MPI_Abort(MPI_COMM_WORLD, ret);
    }
    /* Imprime a versão e subversão da biblioteca MPI */
    MPI_Get_version(&versao,&subversao);
    printf("Versão do MPI  = %d Subversão = %d \n", versao, subversao);
    /* Obtém o ranque e número de processos em execução */
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    /* Define o nome do computador onde o processo está executando */
    MPI_Get_processor_name(maquina, &aux);
    printf("Número de tarefas = %d Meu ranque = %d Executando em  %s\n", num_procs, meu_ranque, maquina);
    /* Finaliza o MPI */
    MPI

In [ ]:
!mpicc -o ./bin/mpi_funcoes ./src/mpi_funcoes.c
!mpirun --allow-run-as-root -np 4 ./bin/mpi_funcoes

Versão do MPI  = 3 Subversão = 1 
Número de tarefas = 4 Meu ranque = 2 Executando em  db785faa09c2
Versão do MPI  = 3 Subversão = 1 
Número de tarefas = 4 Meu ranque = 3 Executando em  db785faa09c2
Versão do MPI  = 3 Subversão = 1 
Número de tarefas = 4 Meu ranque = 0 Executando em  db785faa09c2
Versão do MPI  = 3 Subversão = 1 
Número de tarefas = 4 Meu ranque = 1 Executando em  db785faa09c2


#Medindo o tempo de execução

A função **MPI_Wtime()** retorna (em precisão dupla) o tempo total em segundos decorrido desde um instante determinado no passado. Esse instante é dependente de implementação, mas deve sempre o mesmo para uma dada implementação. A função **MPI_Wtick()** retorna (em precisão dupla) a resolução em segundos  da função **MPI_Wtime()**. Um exemplo de uso dessas funções pode ser visto a seguir.



In [ ]:
!cat src/mpi_wtime.c

#include "mpi.h"
#include <stdio.h>
int main(int argc, char *argv[]) { /* mpi_wtime.c */
  // ...
double tempo_inicial, tempo_final, a;
tempo_inicial = MPI_Wtime();
/* Realiza um trabalho qualquer */
    for (long int i = 0; i < 100000000000; i++) {
	    a = (double) i;
    } 
 // ...
tempo_final = MPI_Wtime();
printf("Foram gastos %3.6f segundos para calcular a = %3.0f com precisão de  %3.3e segundos\n",tempo_final-tempo_inicial, a, MPI_Wtick ());
return(0);
}


In [ ]:
!mpicc -o ./bin/mpi_wtime ./src/mpi_wtime.c
!mpirun --allow-run-as-root -np 2 ./bin/mpi_wtime

Foram gastos 22.563738 segundos para calcular a = 9999999999 com precisão de  1.000e-09 segundos
Foram gastos 22.616066 segundos para calcular a = 9999999999 com precisão de  1.000e-09 segundos


# Exemplo simples

A seguir apresentamos um exemplo simples de programa MPI. Para sua execução o comando a ser utilizado é o *mpicc* que faz a ligação automática com os arquivos de cabeçalho e bibliotecas necessários para a compilação. **MPI_Send()** e **MPI_Recv()** são utilizados, respectivamente, para enviar e receber as mensagens. Reforçando que **MPI_Init()** e **MPI_Finalize()** devem ser utilizados para iniciar e terminar o trecho com chamadas para funções de MPI. Mais uma vez, a função **MPI_Comm_Rank()** retorna o ranque (rank) de um processo no seu segundo argumento. O número de processos executando o programa pode ser obtido com a função **MPI_Comm_size()**.

In [ ]:
!cat ./src/mpi_simples.c

#include <stdio.h>
#include <string.h>
#include "mpi.h"

int main(int argc, char *argv[]) { /* mpi_simples.c  */
int meu_ranque, num_procs;
int origem, destino, etiq = 0;
char mensagem[200];
MPI_Status estado;
    /* Chamada inicial para o MPI */
    MPI_Init(&argc, &argv);
    /* Determina ranque e número de processos em execução */
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    /* Todos os processos com ranque diferente de 0 enviam uma mensagem */
    if (meu_ranque != 0) {
        sprintf(mensagem, "Processo %d está vivo!", meu_ranque);
        destino = 0;
        MPI_Send(mensagem, strlen(mensagem)+1, MPI_CHAR, destino, etiq, MPI_COMM_WORLD);
    } 
    /* Processo com ranque 0 recebe num_procs-1 mensagens */
    else {
        for (origem = 1; origem < num_procs; origem++) {
            MPI_Recv(mensagem, 200, MPI_CHAR, origem, etiq, MPI_COMM_WORLD, &estado);
    /* Imprime as mensagens recebidas */
            printf("%s\n",men

In [ ]:
!mpicc -o bin/mpi_simples src/mpi_simples.c
!mpirun --allow-run-as-root -np 4 ./bin/mpi_simples

Processo 1 está vivo!
Processo 2 está vivo!
Processo 3 está vivo!


A informação sobre a recepção de mensagem com o uso de um coringa  é retornada pela função MPI_Recv em uma estrutura do tipo “MPI_Status”.
Essa estrutura tem diversos usos, por exemplo, para saber o total de elementos recebidos com uso da rotina
**int MPI_Get_count(MPI_Status *status, MPI_Datatype datatype, int *count)**

No programa a seguir, o proceso com ranque 0 envia uma quantidade aleatória de inteiros para o processo 1. Ao receber a mensagem, o processo 1 utilizar a estrutura *estado* (do tipo **MPI_Status**) e a função **MPI_Get_count()** para determinar a quantidade de elementos recebidos na mensagem.

In [ ]:
!cat src/mpi_status.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#define MAX 100

int main(int argc, char *argv[]) { /* mpi_status.c  */
int meu_ranque, total_num, etiq = 0;
int origem=0, destino=1, numeros[MAX];
MPI_Status estado;
    
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    if (meu_ranque == origem) {
    /* Escolhe uma quantidade aleatória de inteiros para enviar para o processo 1 */
        srand(MPI_Wtime());
        total_num = (rand() / (float)RAND_MAX) * MAX;
    /* Envia a quantidade de inteiros para o processo 1 */
        MPI_Send(numeros, total_num, MPI_INT, destino, etiq, MPI_COMM_WORLD);
        printf("Processo %d enviou %d números para 1\n", origem, total_num);
    } 
    else 
        if (meu_ranque == destino) {
    /* Recebe no máximo MAX números do processo 0 */
            MPI_Recv(numeros, MAX, MPI_INT, origem, etiq, MPI_COMM_WORLD, &estado);
    /* Quando chega a mensagem, verifica o status para determinar quantos números foram real

In [ ]:
!mpicc -o bin/mpi_status src/mpi_status.c
!mpirun --allow-run-as-root -np 4 ./bin/mpi_status

Processo 1 recebeu 84 números. Origem da mensagem = 0, etiqueta = 0
Processo 0 enviou 84 números para 1
